In [3]:
import pandas as pd

In [4]:
access = pd.read_csv("/home/jaime/projet_info_3A/private/access.csv",header=0)
id_api = access["Identifiant"][0]
key_api = access["Clé secrète"][0]

In [5]:
import requests 
import os

In [6]:
import emploi_store

In [7]:
client_id = id_api
client_secret = key_api
access_tokens = {}
cached_packages = None
scope = 'api_offresdemploiv2'

In [8]:
# from https://github.com/Cocorico84/employme/blob/855cfcda2c54f42b9c42dcbf4ac87cdf09248b7f/back/manager.py

In [9]:
import requests


url = "https://api.emploi-store.fr/partenaire"

In [10]:
def get_token():
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token'
    r = requests.post(url_token, headers={'Content-Type': 'application/x-www-form-urlencoded'},
                      params={'realm': '/partenaire'},
                      data={'grant_type': 'client_credentials', 'client_id': id_api,
                            'client_secret': key_api,
                            'scope': f'application_{id_api} api_offresdemploiv2 o2dsoffre'})
    data = r.json()
    return data['access_token']


In [35]:
def connection(token,range_str='0-149'):
    url_offers = f'{url}/offresdemploi/v2/offres/search'
    r = requests.get(url=url_offers,params={'range':range_str},   #{'motsCles': 'informatique'} ,# params={'rantge=':range_str}, #,  params={'motsCles': 'informatique'}
        headers={'Authorization': f'Bearer {get_token()}'})
    data = r.json()

    return data

In [74]:
df.columns

Index([               'accessibleTH',                  'alternance',
                'appellationlibelle',                 'competences',
                           'contact',           'dateActualisation',
                      'dateCreation',                 'description',
               'dureeTravailLibelle', 'dureeTravailLibelleConverti',
                        'entreprise',             'experienceExige',
                 'experienceLibelle',                  'formations',
                                'id',                    'intitule',
                           'langues',               'natureContrat',
                      'nombrePostes',           'qualificationCode',
              'qualificationLibelle',    'qualitesProfessionnelles',
                          'romeCode',                 'romeLibelle',
                           'salaire',             'secteurActivite',
            'secteurActiviteLibelle',         'trancheEffectifEtab',
                       'typeContra

In [115]:
def get_results_as_df(range_str='0-149'):
    
    request_as_dict=connection(get_token(),range_str)

    df = pd.DataFrame.from_dict(request_as_dict['resultats'])
    
#     for col_as_list in ['lieuTravail','origineOffre','partenaires']: #, 'competences', 'contact','entreprise','qualitesProfessionnelles','salaire']:
#         if col_as_list in df.columns:
#             df = pd.concat([df.drop([col_as_list], axis=1), df[col_as_list].apply(pd.Series)], axis=1)
    
    return df #request_as_dict['filtres possibles']


In [124]:
# iteration_index = list(range(0,1150,150))
iteration_index = list(range(0,1000,150))

In [125]:
list_dfs = [get_results_as_df(
    str(str(min(i,1000))+'-'+str(min(i+149,1149)))) for i in list(range(0,1150,150))]

In [126]:
all_dfs = pd.concat(list_dfs)

/home/jaime/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [127]:
for col_as_list in ['lieuTravail','origineOffre','partenaires', 'competences', 'contact','entreprise','qualitesProfessionnelles','salaire']:
    if col_as_list in all_dfs.columns:
        all_dfs = pd.concat([all_dfs.drop([col_as_list], axis=1), all_dfs[col_as_list].apply(pd.Series)], axis=1)


In [129]:
all_dfs.iloc[0]

accessibleTH                                                               False
alternance                                                                 False
appellationlibelle                             Ouvrier / Ouvrière en viticulture
dateActualisation                                                            NaN
dateCreation                                           2020-04-05T16:06:05+02:00
deplacementCode                                                              NaN
deplacementLibelle                                                           NaN
description                    Nous recherchons une personne pour effectuer l...
dureeTravailLibelle                                         35H Horaires normaux
dureeTravailLibelleConverti                                          Temps plein
experienceCommentaire                                                        NaN
experienceExige                                                                E
experienceLibelle           

In [130]:
all_dfs = all_dfs.astype(str).drop_duplicates()

In [131]:
all_dfs.to_csv("df_max_pages.csv", sep=",", header=False, index=False)